<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 單純貝氏(Naive Bayes)
描述在已知一些條件下，某事件的發生機率。  
  
**建立前提**：  
每個事件基本上是獨立的，與其他事件沒有關係。


# 理論：建構垃圾郵件篩選器

## 極為簡單的版本 
利用貝氏定理計算「此郵件為垃圾郵件」的機率：
  
在「此郵件包含bitcoin這個字眼」的事件下，此郵件為垃圾郵件的機率是：
```
P(S|B)=[P(B|S)P(S)/[P(B|S)P(S)+P(B|¬S)]P(¬S)
```
* S : 此郵件為垃圾郵件的事件  
* B : 此郵件包含bitcoin這個字眼的事件
  

若手邊有大量郵件確定是垃圾郵件，亦有大量郵件確定不是垃圾郵件。  
可以輕易估算出P(B|S)以及P(B|¬S)的值。  
  
若進一步假設，一封郵件是不是垃圾郵件的機率各占一半。(即P(S)=P(¬S)=0.5)  
則原式可以被改寫為：

```
P(S|B)=[P(B|S)/[P(B|S)P(S)+P(B|¬S)]
```
假設：
* 50%的垃圾郵件含有bitcoin這個字眼
* 1%的非垃圾郵件含有bitcoin這個字眼
→確實是垃圾郵件的機率是：


```
0.5/(0.5+0.01)=98%
```



## 較為精巧的版本

# 實作

# 測試模型

# 運用模型